In [1]:
import numpy as np

In [2]:
def redpr(str):
    print('\x1b[31m' + str + '\x1b[0m')
    
def greenpr(str):
    print('\x1b[32m' + str + '\x1b[0m')
    
def colorpr2(strA, str1, col1, strB, str2, col2):
    print(strA + '\x1b[' + col1 + 'm' + str1.rjust(6) + '\x1b[0m    ', strB.ljust(16) + '\x1b[' + col2 + 'm' + str2.rjust(6) + '\x1b[0m')

In [3]:
week = 14
lastweek = week - 1
wk = str(week)
lwk = str(lastweek)

with open('schedule_2018.txt') as f:
    #passer = f.readlines()
    schedule = list(f)

with open('Week' + lwk + '/SRS_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    SRS = list(f)
    
with open('Week' + lwk + '/YdsOFF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    YdsOFF = list(f)
    
with open('Week' + lwk + '/YdsDEF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    YdsDEF = list(f)
    
with open('Week' + lwk + '/ConvOFF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    ConvOFF = list(f)

with open('Week' + lwk + '/ConvDEF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    ConvDEF = list(f)

In [4]:
team_list = []
data_dict = {}

for i in range(len(SRS)):
    
    # SRS:
    team = SRS[i][:22].lstrip().rstrip()
    team_list.append(team)
    
    val = float(SRS[i].rstrip()[-7:])
    
    data_dict[team] = {}
    data_dict[team]['SRS'] = val
    
data_dict

ind1 = YdsOFF[2].index('G')
ind2 = YdsOFF[2].index('PF')
ind3 = YdsOFF[2].index('Yds')
ind4 = YdsOFF[2].index('Yds', ind3 + 1)
league_PF = float(YdsOFF[36][ind2 - 3:ind2 + 2])
league_yds = float(YdsOFF[36][ind3 - 3:ind3 + 3])
league_pen = float(YdsOFF[36][ind4 - 2:ind4 + 3])
PPY = league_PF/(league_yds)
print(league_PF, league_yds, league_pen, PPY)

for i in range(3, len(SRS) + 3):    
    team = YdsOFF[i][2:25].lstrip()
    G = float(YdsOFF[i].rstrip()[ind1 - 1:ind1 + 1])
    PF1 = float(YdsOFF[i].rstrip()[ind2 - 1:ind2 + 2])
    yds = float(YdsOFF[i].rstrip()[ind3 - 1:ind3 + 3])
    pen = float(YdsOFF[i].rstrip()[ind4 :ind4 + 3])
    yds_tot = yds #- pen
    PF2 = yds_tot*PPY
    PPG1_OFF = PF1/G
    PPG2_OFF = PF2/G
    #print(team, G, PF1, yds, pen, yds_tot, PPG1_OFF, PPG2_OFF)
    data_dict[team]['PPG_OFF'] = [PPG1_OFF, PPG2_OFF]
    data_dict[team]['pen_OFF'] = pen
    #print()
    
    team = YdsDEF[i][2:25].lstrip()
    G = float(YdsDEF[i].rstrip()[ind1 - 1:ind1 + 1])
    PA1 = float(YdsDEF[i].rstrip()[ind2 - 1:ind2 + 2])
    yds = float(YdsDEF[i].rstrip()[ind3 - 1:ind3 + 3])
    pen = float(YdsDEF[i].rstrip()[ind4 :ind4 + 3])
    yds_tot = yds #+ pen
    PA2 = yds_tot*PPY
    PPG1_DEF = PA1/G
    PPG2_DEF = PA2/G
    #print(team, G, PA1, yds, pen, yds_tot, PPG1_DEF, PPG2_DEF)
    data_dict[team]['PPG_DEF'] = [PPG1_DEF, PPG2_DEF]
    data_dict[team]['pen_DEF'] = pen
    #print()
    
    team = ConvOFF[i][2:25].lstrip()
    DwnPct = float(ConvOFF[i].rstrip()[26:30])
    RZPct = float(ConvOFF[i].rstrip()[32:36])
    data_dict[team]['DwnPct_OFF'] = DwnPct
    data_dict[team]['RZPct_OFF'] = RZPct
    #print(DwnPct, RZPct)
    team = ConvDEF[i][2:25].lstrip()
    DwnPct = float(ConvDEF[i].rstrip()[26:30])
    RZPct = float(ConvDEF[i].rstrip()[32:36])
    data_dict[team]['DwnPct_DEF'] = DwnPct
    data_dict[team]['RZPct_DEF'] = RZPct
    #print(DwnPct, RZPct)
    #print()
      
team_list = sorted(team_list)

9186.0 137987.0 21893.0 0.06657148861849305


In [11]:
SRS_ar = []
OFF_ar = []
DEF_ar = []
pow_ar = []
coaching = np.ndarray((32, 6))

i = 0
for team in team_list:
    coaching[i, 0] = data_dict[team]['pen_OFF']
    coaching[i, 1] = data_dict[team]['DwnPct_OFF']
    coaching[i, 2] = data_dict[team]['RZPct_OFF']
    coaching[i, 3] = data_dict[team]['pen_DEF']
    coaching[i, 4] = data_dict[team]['DwnPct_DEF']
    coaching[i, 5] = data_dict[team]['RZPct_DEF']
    i += 1

maxvals = np.amax(coaching, axis=0)
minvals = np.amin(coaching, axis=0)
centers = (maxvals + minvals)/2
sizes = (maxvals - minvals)/4

coaching2 = coaching.copy()
i = 0    
for team in team_list:
    coaching2[i, :] = (coaching2[i, :] - centers)/sizes
    coaching2[i, 0] *= -1.
    coaching2[i, 3] *= -1.
    coaching2[i, 4] *= -1.
    coaching2[i, 5] *= -1.
    
    data_dict[team]['coaching_OFF'] = np.mean(coaching2[i, 0:3])
    data_dict[team]['coaching_DEF'] = np.mean(coaching2[i, 3:6])
    
    SRS_val = data_dict[team]['SRS']
    rnk_1 = data_dict[team]['PPG_OFF'][0]
    rnk_2 = data_dict[team]['PPG_OFF'][1]
    OFF_val0 = np.mean([rnk_1, rnk_2]) 
    OFF_val = OFF_val0 + data_dict[team]['coaching_OFF']
    rnk_3 = data_dict[team]['PPG_DEF'][0]
    rnk_4 = data_dict[team]['PPG_DEF'][1]
    DEF_val0 = np.mean([rnk_3, rnk_4]) 
    DEF_val = DEF_val0 - data_dict[team]['coaching_DEF']
    
    i += 1
    
    SRS_ar.append([SRS_val, team]) 
    OFF_ar.append([OFF_val, team, rnk_1, rnk_2, OFF_val0, data_dict[team]['coaching_OFF']])
    DEF_ar.append([DEF_val, team, rnk_3, rnk_4, DEF_val0, data_dict[team]['coaching_DEF']])
    pow_ar.append([np.mean([SRS_val, OFF_val - DEF_val]), team, SRS_val, OFF_val - DEF_val])
    
    
SRS_rankings = sorted(SRS_ar, reverse=True)
power_OFF = sorted(OFF_ar, reverse=True)
power_DEF = sorted(DEF_ar, reverse=False)
power_rankings = sorted(pow_ar, reverse=True)



print('Power rankings:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), power_rankings[i][1].rjust(22), '{0:+5.1f} {1:+5.1f} {2:+5.1f}'.format(power_rankings[i][0], power_rankings[i][2], power_rankings[i][3]))
    
print()
print('SRS:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), SRS_rankings[i][1].rjust(22), '{0:+5.1f}'.format(SRS_rankings[i][0] ))
    
print()
print('Offense:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), power_OFF[i][1].rjust(22), '{0:4.1f}'.format(power_OFF[i][0]), '   {}'.format(power_OFF[i][2:]))
    team = power_OFF[i][1].lstrip().rstrip()
    data_dict[team]['OFFENSE'] = [i+1, power_OFF[i][0]]
    
print()
print('Defense:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), power_DEF[i][1].rjust(22), '{0:4.1f}'.format(power_DEF[i][0]), '   {}'.format(power_DEF[i][2:]))
    team = power_DEF[i][1].lstrip().rstrip()
    data_dict[team]['DEFENSE'] = [i+1, power_DEF[i][0]]

Power rankings:

 1     New Orleans Saints +10.9 +12.5  +9.2
 2       Los Angeles Rams  +9.6 +10.9  +8.3
 3     Kansas City Chiefs  +7.9 +10.1  +5.8
 4       Baltimore Ravens  +7.0  +6.6  +7.4
 5   Los Angeles Chargers  +6.7  +6.6  +6.7
 6    Pittsburgh Steelers  +6.6  +6.8  +6.3
 7          Chicago Bears  +5.7  +5.2  +6.2
 8   New England Patriots  +4.9  +4.8  +4.9
 9       Seattle Seahawks  +4.3  +5.9  +2.6
10         Houston Texans  +4.0  +4.1  +3.9
11         Dallas Cowboys  +2.5  +2.2  +2.8
12     Indianapolis Colts  +2.0  +0.5  +3.6
13         Denver Broncos  +1.8  +3.6  -0.1
14      Minnesota Vikings  +1.5  -0.3  +3.2
15      Carolina Panthers  +0.6  -0.4  +1.7
16      Green Bay Packers  +0.0  -1.6  +1.6
17    Philadelphia Eagles  -0.5  -1.4  +0.4
18       Tennessee Titans  -1.2  -1.5  -0.8
19   Tampa Bay Buccaneers  -1.6  -2.9  -0.3
20        Atlanta Falcons  -1.7  -1.5  -2.0
21   Jacksonville Jaguars  -1.8  -3.2  -0.5
22    Washington Redskins  -2.3  -2.0  -2.6
23       Clevel

In [21]:
for i in range(len(schedule)):
    if ( schedule[i][0]==wk and schedule[i][1]==' ' ) or schedule[i][0:2]==wk:
        
        team_away = schedule[i][3:29].lstrip().rstrip()
        team_home = schedule[i][28:].lstrip().rstrip()
        
        spread1 = 2.5 + (- data_dict[team_away]['SRS'] + data_dict[team_home]['SRS']) / 2.
        
        rnk_1 = data_dict[team_away]['OFFENSE'][0]
        col_1 = '30'
        if rnk_1 <= 8:
            col_1 = '32'
        elif rnk_1 >= 25:
            col_1 = '31'
        str_1 = str(rnk_1).rjust(3)
        
        rnk_2 = data_dict[team_home]['DEFENSE'][0]
        col_2 = '30'
        if rnk_2 <= 8:
            col_2 = '32'
        elif rnk_2 >= 25:
            col_2 = '31'
        str_2 = str(rnk_2).rjust(3)
        
        rnk_3 = data_dict[team_away]['DEFENSE'][0]
        col_3 = '30'
        if rnk_3 <= 8:
            col_3 = '32'
        elif rnk_3 >= 25:
            col_3 = '31'
        str_3 = str(rnk_3).rjust(3)
        
        rnk_4 = data_dict[team_home]['OFFENSE'][0]
        col_4 = '30'
        if rnk_4 <= 8:
            col_4 = '32'
        elif rnk_4 >= 25:
            col_4 = '31'
        str_4 = str(rnk_4).rjust(3)
        
        OFF_away = data_dict[team_away]['OFFENSE'][1]        
        OFF_home = data_dict[team_home]['OFFENSE'][1]
        
        DEF_away = data_dict[team_away]['DEFENSE'][1]
        DEF_home = data_dict[team_home]['DEFENSE'][1]
        
        Pts_away = np.mean([OFF_away, DEF_home])
        Pts_home = np.mean([OFF_home, DEF_away])
        
        spread2 = 2.5 + ( - Pts_away + Pts_home)
        
        spread = (spread1 + spread2) / 2
        total = Pts_away + Pts_home
        score_away = total/2 - spread/2
        score_home = total/2 + spread/2
        
        #print(team_away.rjust(22) + ' {0:+5.1f} ({1:+5.1f}, {2:+5.1f})   at   '.format(spread, spread1, spread2) + team_home )
        print(team_away.rjust(22) + ' {0:+5.1f}  at  '.format(np.round(2*spread)/2) + team_home)  
        print('                       {0:2.0f}:{1:2.0f}'.format(np.round(score_away), np.round(score_home)))
        print('                total: {}'.format(np.round(2*total)/2))
        #print('penalties:  {0:3d}'.format(data_dict[team_away]['penalties'][1]).rjust(22))
        colorpr2('OFFENSE:      ', str_1, col_1, '   DEFENSE:      ', str_2, col_2)
        colorpr2('DEFENSE:      ', str_3, col_3, '   OFFENSE:      ', str_4, col_4)
        #colorpr2('timessacked:      ', str_5, col_5, '   sacks:', str_8, col_8)
        #colorpr2('rushOFF:          ', str_9, col_9, '   rushDEF:', str_12, col_12)
        #colorpr2('3DconvOFF:        ', str_13, col_13, '   3DconvDEF:', str_16, col_16)
        #colorpr2('redzoneOFF:       ', str_17, col_17, '   redzoneDEF:', str_20, col_20)
        #print('----------------------------------------------------------')
        #print('OFFENSE:             {0:3d} ({1:4.1f})'.format(data_dict[team_away]['OFFENSE'][0], data_dict[team_away]['OFFENSE'][1]), \
        #      '  DEFENSE:      {0:3d} ({1:4.1f})'.format(data_dict[team_home]['DEFENSE'][0], data_dict[team_home]['DEFENSE'][1]))
        print()
        #colorpr2('interceptDEF:     ', str_1, col_1, '   interceptOFF:', str_4, col_4)
        #colorpr2('sacks:            ', str_7, col_7, '   timessacked:', str_6, col_6)
        #colorpr2('rushDEF:          ', str_11, col_11, '   rushOFF:', str_10, col_10)
        #colorpr2('3DconvDEF:        ', str_15, col_15, '   3DconvOFF:', str_14, col_14)
        #colorpr2('redzoneDEF:       ', str_19, col_19, '   redzoneOFF:', str_18, col_18)
        #print('----------------------------------------------------------')
        #print('DEFENSE:             {0:3d} ({1:4.1f})'.format(data_dict[team_away]['DEFENSE'][0], data_dict[team_away]['DEFENSE'][1]), \
        #      '  OFFENSE:      {0:3d} ({1:4.1f})'.format(data_dict[team_home]['OFFENSE'][0], data_dict[team_home]['OFFENSE'][1]))
        #print()
        print()

  Jacksonville Jaguars  +3.0  at  Tennessee Titans
                       18:21
                total: 40.0
OFFENSE:          30        DEFENSE:           7
DEFENSE:           3        OFFENSE:          27


      Baltimore Ravens  +3.0  at  Kansas City Chiefs
                       25:28
                total: 52.5
OFFENSE:          12        DEFENSE:          28
DEFENSE:           1        OFFENSE:           1


       New York Giants  +3.0  at  Washington Redskins
                       21:24
                total: 45.0
OFFENSE:          22        DEFENSE:          14
DEFENSE:          24        OFFENSE:          26


    New Orleans Saints  -3.5  at  Tampa Bay Buccaneers
                       30:26
                total: 56.0
OFFENSE:           3        DEFENSE:          31
DEFENSE:          11        OFFENSE:           5


      Los Angeles Rams  +0.5  at  Chicago Bears
                       26:26
                total: 52.0
OFFENSE:           2        DEFENSE:           4
DEFEN

In [8]:
np.round?